In [1]:
import pandas as pd

In [5]:
lyrics = pd.read_csv('../genius_lyrics.csv')

In [12]:
from gensim.utils import simple_preprocess

C:\Users\zylst\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [18]:
with open('../lyrics.txt') as f:
    lyrics = f.readlines()

In [19]:
lyrics

['0       Where is the moment we needed the most?\\nYou k...\n',
 '1       The gyal dem Schillaci, Sean da Paul\\nSo me gi...\n',
 "2       Am I throwin' you off?\\nNope\\nDidn't think so\\...\n",
 '3       My life is brilliant\\nMy life is brilliant, my...\n',
 '4       Ladies up in here tonight\\nNo fighting (We got...\n',
 '                              ...                        \n',
 "1392    We've been to both Carolinas\\nSeen a big Monta...\n",
 "1393    I'm jealous of the blue jeans that you're wear...\n",
 "1394    I'm a motherfuckin' train wreck\\nI don't wanna...\n",
 "1395    There's somethin' in the way you roll your eye...\n",
 '1396    Man, what? (Haha)\\nThis shit funny, one sec\\nO...\n',
 'Length: 1397, dtype: object']

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

string = 

### I want to see how wordy/lengthy these songs are
Could possibly run it with length of song to see just how much message is getting packed into there.<br>
Message = not repetitive, lots of words per ms?

In [13]:
#Character count
char_count = []

for i in range(0,496):
    chars = len(lyrics['0'][i])
    char_count.append(chars) 

In [14]:
# word count
def count_words(string):
    words = string.split()
    return len(words)